### Checkpoint 2 - Data Science - 2ESA

Cassio Valezzi - 551059        
Lucca Sabatini - 98169        
Victor Nuzzi   - 98209 

### Fazer instalação das bibliotecas

In [19]:
%pip install pandas seaborn matplotlib plotly statsmodels jinja2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Fazer importação das bibliotecas que serão usadas

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
from io import BytesIO
import base64
from jinja2 import Template

### Função para salvar gráficos como Base64

In [21]:
def plot_to_base64(plt):
    img = BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    return base64.b64encode(img.getvalue()).decode('utf-8')

### Função principal para fazer analises e depois ser importada para o HTML

In [22]:
def analyze_dataframe(df, output_file="report.html"):
    sns.set(style="whitegrid")

    #Estatísticas descritivas
    stats = df.describe().T
    
    #Apenas colunas numéricas para a correlação
    numerical_df = df.select_dtypes(include=['float64', 'int64'])
    
    #Gráfico de correlação
    if not numerical_df.empty:
        corr = numerical_df.corr()
        plt.figure(figsize=(10, 6))
        sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5)
        plt.title('Correlation Heatmap')
        corr_img = plot_to_base64(plt)
        plt.close()
    else:
        corr_img = None
    
    #Histogramas de cada coluna
    histograms = {}
    for column in numerical_df.columns:
        if column != "A_id":
            plt.figure(figsize=(6, 4))
            sns.histplot(df[column], kde=True)
            plt.title(f'Histogram of {column}')
            histograms[column] = plot_to_base64(plt)
            plt.close()
    
    #Scatter plots
    scatter_plots = {}
    numerical_columns = numerical_df.columns
    for i, col1 in enumerate(numerical_columns):
        if col1 != "A_id":
            for col2 in numerical_columns[i+1:]:
                plt.figure(figsize=(6, 4))
                sns.scatterplot(x=col1, y=col2, data=df)
                plt.title(f'Scatter plot between {col1} and {col2}')
                scatter_plots[f'{col1}_vs_{col2}'] = plot_to_base64(plt)
                plt.close()

    #Regressão linear
    regression_plots = {}
    for col1 in numerical_columns:
        if col1 != "A_id":
            for col2 in numerical_columns:
                    if (col1 != col2) and col2 != "A_id":
                        #Arrumar valores
                        subset_df = df[[col1, col2]].dropna()
                        
                        if not subset_df.empty:
                            X = sm.add_constant(subset_df[col1])
                            Y = subset_df[col2]
                            model = sm.OLS(Y, X).fit()
                            plt.figure(figsize=(6, 4))
                            sns.jointplot(x=col1, y=col2, data=subset_df,kind="reg", truncate=False,color="m")
                            # plt.title(f'Regression: {col1} vs {col2}')
                            regression_plots[f'Regression: {col1} vs {col2}'] = plot_to_base64(plt)
                            plt.close()

    #HTML
    template = """
    <html>
    <head>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
        <title>Data Analysis Report</title>
        <style>
            body {
                padding-top: 20px;
            }
            h1, h2 {
                text-align: center;
                margin-bottom: 30px;
            }
            .section {
                margin-bottom: 50px;
            }
            .img-fluid {
                margin: 20px 0;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1 class="mt-5">Automated Data Analysis Report</h1>
            <p class="text-center">
            This report provides a detailed visual and statistical analysis of the given dataset, using various techniques like correlation heatmaps, histograms, scatter plots, and linear regressions.
            </p>
           
            <div class="section">
                <h2>1. Correlação Heatmap</h2>
                <p>
                Este mapa de calor mostra a correlação entre colunas numéricas no conjunto de dados. Um valor próximo de 1 indica uma forte correlação positiva, enquanto um valor próximo de -1 indica uma forte correlação negativa. Um valor próximo de 0 significa que não há correlação.
                </p>
                {% if corr_img %}
                <img src="data:image/png;base64,{{ corr_img }}" class="img-fluid" alt="Correlation Heatmap">
                {% else %}
                <p>No numerical data available for correlation analysis.</p>
                {% endif %}
            </div>
            
            <div class="section">
                <h2>2. Estatisticas Descritivas</h2>
                <p>
                A tabela abaixo resume as estatísticas básicas para cada coluna numérica, incluindo a contagem, média, desvio padrão e os valores mínimo e máximo.
                </p>
                <table class="table table-bordered">
                    <thead>
                        <tr>
                            <th>Column</th>
                            <th>Count</th>
                            <th>Mean</th>
                            <th>Std</th>
                            <th>Min</th>
                            <th>25%</th>
                            <th>50%</th>
                            <th>75%</th>
                            <th>Max</th>
                        </tr>
                    </thead>
                    <tbody>
                        {% for col, stats in stats.iterrows() %}
                        <tr>
                            <td>{{ col }}</td>
                            <td>{{ stats['count'] }}</td>
                            <td>{{ stats['mean'] }}</td>
                            <td>{{ stats['std'] }}</td>
                            <td>{{ stats['min'] }}</td>
                            <td>{{ stats['25%'] }}</td>
                            <td>{{ stats['50%'] }}</td>
                            <td>{{ stats['75%'] }}</td>
                            <td>{{ stats['max'] }}</td>
                        </tr>
                        {% endfor %}
                    </tbody>
                </table>
            </div>
            
            <div class="section">
                <div class="container text-center">
                    <h2>3. Histograms</h2>
                    <p>
                    Os gráficos de dispersão mostram a relação entre duas variáveis numéricas. Um padrão pode ajudar a identificar correlações, tendências ou valores discrepantes.
                    </p>
                    <div class="row row-cols-2">
                        {% for col, img in histograms.items() %}
                        <div class="col">
                        <h3>{{ col }}</h3>
                        <img src="data:image/png;base64,{{ img }}" class="img-fluid" alt="Histogram for {{ col }}">
                        </div>
                        {% endfor %}
                    </div>
                </div>
            </div>
            
            <div class="section">
                <div class="container text-center">
                    <h2>4. Scatter Plots</h2>
                    <p>
                    Os gráficos de dispersão mostram a relação entre duas variáveis numéricas. Um padrão pode ajudar a identificar correlações, tendências ou valores discrepantes.
                    </p>
                    <div class="row row-cols-2">
                        {% for title, img in scatter_plots.items() %}
                        <div class="col">
                        <h3>{{ title }}</h3>
                        <img src="data:image/png;base64,{{ img }}" class="img-fluid" alt="{{ title }}">
                        </div>
                        {% endfor %}
                    </div>
                </div> 
            </div>
            
            <div class="section">
                <div class="container text-center">
                    <h2>5. Regressão Linear</h2>
                    <p>
                    A regressão linear tenta modelar a relação entre duas variáveis ​​numéricas ajustando uma linha através dos pontos de dados. A linha vermelha nos gráficos representa o ajuste linear.
                    </p>
                    <div class="row row-cols-2">                                          
                        {% for title, img in regression_plots.items() %}
                        <div class="col">
                        <h3>{{ title }}</h3>
                        <img src="data:image/png;base64,{{ img }}" class="img-fluid" alt="Regression plot for {{ title }}">
                        </div>
                        {% endfor %}
                    </div>
                </div>
            </div>
        </div>
    </body>
    </html>
    """
    
    html_content = Template(template).render(
        corr_img=corr_img, 
        stats=stats, 
        histograms=histograms, 
        scatter_plots=scatter_plots, 
        regression_plots=regression_plots
    )
    
    # Salvar o relatório em HTML com codificação UTF-8
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(html_content)
    print(f"Report saved to {output_file}")



### Carregar o dataset e depois criar o arquivo HTML 

In [23]:
#Carregar dataset
df = pd.read_csv('apple_quality.csv')

#Rodar a análise e gerar o arquivo HTML
analyze_dataframe(df, output_file="apple_quality_report.html")

C:\Users\lucca\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\seaborn\axisgrid.py:1696: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  f = plt.figure(figsize=(height, height))
C:\Users\lucca\AppData\Local\Temp\ipykernel_10672\1390726002.py:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(6, 4))


Report saved to apple_quality_report.html


<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

...

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>